In [1]:
import pandas as pd

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-16 04:04:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-06-16 04:04:01 (6.56 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://lgodleski-bucket.s3.us-east-2.amazonaws.com/murders_county_merged.csv"
spark.sparkContext.addFile(url)
merged_data_df = spark.read.csv(SparkFiles.get("murders_county_merged.csv"), sep=",", header=True, inferSchema=True)

In [6]:
merged_data_df.show()

+--------+-------+--------+------+------+--------+-----+-----+------+-----+-------+-------+------+---------+------------+---------------+-------+------------+--------+------------+
|censusid| county|totalpop|   men| women|hispanic|white|black|native|asian|pacific|citizen|income|incomeerr|incomepercap|incomepercaperr|poverty|childpoverty|employed|unemployment|
+--------+-------+--------+------+------+--------+-----+-----+------+-----+-------+-------+------+---------+------------+---------------+-------+------------+--------+------------+
|    6001|Alameda| 1584983|776699|808284|    22.6| 33.0| 11.3|   0.3| 27.5|    0.8|1025865| 75619|      613|       37285|            279|   12.5|        15.2|  778132|         8.3|
|    6001|Alameda| 1584983|776699|808284|    22.6| 33.0| 11.3|   0.3| 27.5|    0.8|1025865| 75619|      613|       37285|            279|   12.5|        15.2|  778132|         8.3|
|    6001|Alameda| 1584983|776699|808284|    22.6| 33.0| 11.3|   0.3| 27.5|    0.8|1025865| 756

In [7]:
# Add URL for murders_cleaned dataset
from pyspark import SparkFiles
url2 = "https://lgodleski-bucket.s3.us-east-2.amazonaws.com/murders_clean.csv"
spark.sparkContext.addFile(url2)
murders_df = spark.read.csv(SparkFiles.get("murders_clean.csv"), sep=",", header=True, inferSchema=True)


In [8]:
# Add URL for county_cleaned dataset
from pyspark import SparkFiles
url3 = "https://lgodleski-bucket.s3.us-east-2.amazonaws.com/county_clean.csv"
spark.sparkContext.addFile(url3)
county_df = spark.read.csv(SparkFiles.get("county_clean.csv"), sep=",", header=True, inferSchema=True)
